In [1]:
#imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
from sys import platform
import io
import datetime

if platform == "linux" or platform == "linux2":
    data_string = "/home/maria/Documents/data/pied_piper/puyallup/"
    output_string = "/home/maria/Documents/output/pied_piper/puyallup/"

elif platform == "win32"or platform == "win64":
    data_string = "C:/Users/maria/OneDrive/Documents/data/pied_piper/puyallup/"
    output_string = "C:/Users/maria/OneDrive/Documents/output/pied_piper/puyallup/"

In [8]:
# #read file in Windows
try:
    df = pd.read_excel(
   io=data_string + "puyallup_chinook_2004-2018.xlsx",
   sheet_name ='2011_edited',
    header = [3]
)
except PermissionError:
    print("Permission Error. Close the file, if it is open.")
    

df.head()

,Date,flow,In_day,secchi_depth_day,turbidity,chinook0_wild_num_day,Unnamed: 6,Unnamed: 7,In_night,secchi_depth_night,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,chinook0_hatchery_num_day,Unnamed: 26,Unnamed: 27,chinook0_hatchery_num_night,Unnamed: 29,chinook0_hatchery_num
0,2011-02-03,1880,NaN,NaN,NaN,NaN,NaN,NaN,14.5,148,...,2011-02-03,1880,0,0,-,14.5,148,0,14.50,0
1,2011-02-04,2340,7.25,205,NaN,4.0,NaN,NaN,14.5,115,...,2011-02-04,2340,7.25,205,0,14.5,115,0,21.75,0
2,2011-02-05,2670,10,120,NaN,6.0,NaN,NaN,14.25,154,...,2011-02-05,2670,10,120,0,14.25,154,0,24.25,0
3,2011-02-06,2410,8.75,150,NaN,4.0,NaN,NaN,14.75,120,...,2011-02-06,2410,8.75,150,0,14.75,120,0,23.50,0
4,2011-02-07,2670,8.25,130,NaN,0.0,NaN,NaN,14.5,148,...,2011-02-07,2670,8.25,130,0,14.5,148,0,22.75,0


In [9]:
pd.unique(df['In_day']) # 'OUT',nan, 'ST'
pd.unique(df['In_night'])# 'OUT'

df.columns

Index(['Date', 'flow', 'In_day', 'secchi_depth_day', 'turbidity',
       'chinook0_wild_num_day', 'Unnamed: 6', 'Unnamed: 7', 'In_night',
       'secchi_depth_night', 'Unnamed: 10', 'chinook0_wild_num_night',
       'Unnamed: 12', 'Unnamed: 13', 'In', 'secchi_depth', 'chinook0_wild_num',
       'chinook0_wild_num_estimate', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'chinook0_hatchery_num_day', 'Unnamed: 26',
       'Unnamed: 27', 'chinook0_hatchery_num_night', 'Unnamed: 29',
       'chinook0_hatchery_num'],
      dtype='object')

In [10]:
df_subset = df[['Date', 'flow', 'secchi_depth_day','In_day','In_night','In', 'secchi_depth_night',
                'chinook0_wild_num_day',
                'chinook0_wild_num_night','chinook0_wild_num',
                'chinook0_hatchery_num_day',
                'chinook0_hatchery_num_night','chinook0_hatchery_num',
                'chinook0_wild_num_estimate']]

In [11]:
df_subset = df_subset.replace({'-':np.nan})
df_subset = df_subset.replace({'OUT':np.nan})
df_subset = df_subset.replace({'ST':np.nan})
df_subset.loc[pd.isna(df_subset['In_day']),'chinook0_wild_num_day'] =np.nan
# df_subset.loc[pd.isna(df_subset['In_day']),'chinook0_wild_num_day_estimate'] =np.nan
df_subset.loc[pd.isna(df_subset['In_day']),'chinook0_hatchery_num_day'] =np.nan

df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_wild_num_night'] =np.nan
# df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_wild_num_night_estimate'] =np.nan
df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_hatchery_num_night'] =np.nan

In [12]:
#if trap was out for both day and night, the total values should be zero
df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_hatchery_num'][pd.isna(df_subset['In_day'])] = np.nan
df_subset.loc[pd.isna(df_subset['In_night']),'chinook0_wild_num'][pd.isna(df_subset['In_day'])] = np.nan

In [13]:
df_subset['chinook0_wild_perhour_day'] = df_subset['chinook0_wild_num_day']/df_subset['In_day']
df_subset['chinook0_wild_perhour_night'] = df_subset['chinook0_wild_num_night']/df_subset['In_night']
df_subset['chinook0_wild_perhour'] = df_subset['chinook0_wild_num']/df_subset['In']

df_subset['chinook0_hatchery_perhour_day'] = df_subset['chinook0_hatchery_num_day']/df_subset['In_day']
df_subset['chinook0_hatchery_perhour_night'] = df_subset['chinook0_hatchery_num_night']/df_subset['In_night']
df_subset['chinook0_hatchery_perhour'] = df_subset['chinook0_hatchery_num']/df_subset['In']

# df_subset['secchi_depth_day'] = df_subset['secchi_depth']

df_subset['flow_day'] = df_subset['flow']
df_subset['flow_night'] = df_subset['flow']

In [14]:
df_subset['trap_efficiency'] = df_subset['chinook0_wild_num']/df_subset['chinook0_wild_num_estimate']

df_subset['chinook0_wild_perhour_day_estimate'] = (df_subset['chinook0_wild_perhour_day']/
                                                   df_subset['trap_efficiency'])
df_subset['chinook0_wild_perhour_night_estimate'] = (df_subset['chinook0_wild_perhour_night']/
                                                   df_subset['trap_efficiency'])
df_subset['chinook0_wild_perhour_estimate'] = (df_subset['chinook0_wild_perhour']/
                                                   df_subset['trap_efficiency'])

df_subset['chinook0_wild_num_day_estimate'] = df_subset['chinook0_wild_num_day']/df_subset['trap_efficiency']
df_subset['chinook0_wild_num_night_estimate'] = df_subset['chinook0_wild_num_night']/df_subset['trap_efficiency']


df_subset['chinook0_hatchery_num_day_estimate'] = df_subset['chinook0_hatchery_num_day']/df_subset['trap_efficiency']
df_subset['chinook0_hatchery_num_night_estimate'] = df_subset['chinook0_hatchery_num_night']/df_subset['trap_efficiency']


df_subset['chinook0_hatchery_perhour_day_estimate'] = (df_subset['chinook0_hatchery_num_day_estimate']/
                                                   df_subset['In_day'])
df_subset['chinook0_hatchery_perhour_night_estimate'] = (df_subset['chinook0_hatchery_num_night_estimate']/
                                                   df_subset['In_night'])
df_subset['chinook0_hatchery_perhour_estimate'] = ((df_subset['chinook0_hatchery_num_day_estimate']+df_subset['chinook0_hatchery_num_night_estimate'])/
                                                   df_subset['In'])

In [15]:
df_subset['chinook0_wild_day_fraction'] = np.nan
df_subset['chinook0_hatchery_day_fraction'] = np.nan
n = len(df_subset['chinook0_wild_perhour'])

for i in range(n):
    if df_subset['chinook0_wild_perhour_day'].loc[i]==0 and df_subset['chinook0_wild_perhour_night'].loc[i] == 0:
        df_subset['chinook0_wild_day_fraction'].loc[i] = np.nan
        
    else:
        df_subset['chinook0_wild_day_fraction'].loc[i] = (df_subset['chinook0_wild_perhour_day'].loc[i]/
                                                          (df_subset['chinook0_wild_perhour_day'].loc[i] + df_subset['chinook0_wild_perhour_night'].loc[i]))
        
    if df_subset['chinook0_hatchery_perhour_day'].loc[i]==0 and df_subset['chinook0_hatchery_perhour_night'].loc[i] == 0:
        df_subset['chinook0_hatchery_day_fraction'].loc[i] = np.nan
        
    else:
        df_subset['chinook0_hatchery_day_fraction'].loc[i] = (df_subset['chinook0_hatchery_perhour_day'].loc[i]/
                                                          (df_subset['chinook0_hatchery_perhour_day'].loc[i] + df_subset['chinook0_hatchery_perhour_night'].loc[i]))
        
        

/home/maria/miniconda3/envs/analysis/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_subset)

,Date,flow,secchi_depth_day,In_day,In_night,In,secchi_depth_night,chinook0_wild_num_day,chinook0_wild_num_night,chinook0_wild_num,chinook0_hatchery_num_day,chinook0_hatchery_num_night,chinook0_hatchery_num,chinook0_wild_num_estimate,chinook0_wild_perhour_day,chinook0_wild_perhour_night,chinook0_wild_perhour,chinook0_hatchery_perhour_day,chinook0_hatchery_perhour_night,chinook0_hatchery_perhour,flow_day,flow_night,trap_efficiency,chinook0_wild_perhour_day_estimate,chinook0_wild_perhour_night_estimate,chinook0_wild_perhour_estimate,chinook0_wild_num_day_estimate,chinook0_wild_num_night_estimate,chinook0_hatchery_num_day_estimate,chinook0_hatchery_num_night_estimate,chinook0_hatchery_perhour_day_estimate,chinook0_hatchery_perhour_night_estimate,chinook0_hatchery_perhour_estimate,chinook0_wild_day_fraction,chinook0_hatchery_day_fraction
0,2011-02-03,1880,NaN,NaN,14.50,14.50,148.0,NaN,1.0,1,NaN,0.0,0,11.651710,NaN,0.068966,0.068966,NaN,0.000000,0.000000,1880,1880,0.085824,NaN,0.803566,0.803566,NaN,11.651710,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN
1,2011-02-04,2340,205.0,7.25,14.50,21.75,115.0,4.0,1.0,5,0.0,0.0,0,58.258548,0.551724,0.068966,0.229885,0.000000,0.000000,0.000000,2340,2340,0.085824,6.428529,0.803566,2.678554,46.606838,11.651710,0.000000,0.000000,0.000000,0.000000,0.000000,0.888889,NaN
2,2011-02-05,2670,120.0,10.00,14.25,24.25,154.0,6.0,3.0,9,0.0,0.0,0,104.865386,0.600000,0.210526,0.371134,0.000000,0.000000,0.000000,2670,2670,0.085824,6.991026,2.452991,4.324346,69.910257,34.955129,0.000000,0.000000,0.000000,0.000000,0.000000,0.740260,NaN
3,2011-02-06,2410,150.0,8.75,14.75,23.50,120.0,4.0,6.0,10,0.0,0.0,0,116.517096,0.457143,0.406780,0.425532,0.000000,0.000000,0.000000,2410,2410,0.085824,5.326496,4.739678,4.958174,46.606838,69.910257,0.000000,0.000000,0.000000,0.000000,0.000000,0.529148,NaN
4,2011-02-07,2670,130.0,8.25,14.50,22.75,148.0,0.0,14.0,14,0.0,0.0,0,163.123934,0.000000,0.965517,0.615385,0.000000,0.000000,0.000000,2670,2670,0.085824,0.000000,11.249926,7.170283,0.000000,163.123934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
5,2011-02-08,2480,178.0,8.00,15.50,23.50,181.0,23.0,20.0,43,0.0,0.0,0,501.023511,2.875000,1.290323,1.829787,0.000000,0.000000,0.000000,2480,2480,0.085824,33.498665,15.034464,21.320149,267.989320,233.034191,0.000000,0.000000,0.000000,0.000000,0.000000,0.690223,NaN
6,2011-02-09,2200,172.0,7.75,14.50,22.25,182.0,4.0,14.0,18,0.0,0.0,0,209.730772,0.516129,0.965517,0.808989,0.000000,0.000000,0.000000,2200,2200,0.085824,6.013786,11.249926,9.426102,46.606838,163.123934,0.000000,0.000000,0.000000,0.000000,0.000000,0.348348,NaN
7,2011-02-10,2000,183.0,9.50,14.25,23.75,187.0,3.0,3.0,6,0.0,0.0,0,69.910257,0.315789,0.210526,0.252632,0.000000,0.000000,0.000000,2000,2000,0.085824,3.679487,2.452991,2.943590,34.955129,34.955129,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,NaN
8,2011-02-11,1900,185.0,9.75,13.75,23.50,185.0,1.0,4.0,5,0.0,0.0,0,58.258548,0.102564,0.290909,0.212766,0.000000,0.000000,0.000000,1900,1900,0.085824,1.195047,3.389588,2.479087,11.651710,46.606838,0.000000,0.000000,0.000000,0.000000,0.000000,0.260664,NaN
9,2011-02-12,1850,90.0,9.75,13.25,23.00,100.0,1.0,11.0,12,0.0,0.0,0,139.820515,0.102564,0.830189,0.521739,0.000000,0.000000,0.000000,1850,1850,0.085824,1.195047,9.673117,6.079153,11.651710,128.168805,0.000000,0.000000,0.000000,0.000000,0.000000,0.109959,NaN


In [17]:
#save the data

df_subset.to_csv(data_string + "2011_puyallup.csv")  